# Preguntas
- Cuánto pagan de expensas los departamentos en CABA según barrio?
  - La media de las expensas según barrio
  - Media del ratio entre alquiler y expensas
  - media de expensas por m2

Para ello:
- Método para detectar outliers
- Método para detectar no vivienda

In [26]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np
import re

from config.config_jupyter import set_wd
set_wd()

In [27]:
palabras_identifican_no_vivienda = ["oficina", "local", "oficinas", "galpón"]

In [28]:
df = pd.read_parquet("output/zonaprop.pkt")

In [29]:
conditions_currency = [
    df['Price'].str.startswith('USD'),
    df['Price'].str.startswith('$') | df["Price"].str.startswith("Pesos"),
    df['Price'].str.startswith('Consultar')
]

values = ['USD', 'ARS', 'Consultar']

df['currency'] = np.select(conditions_currency, values, default=np.nan)


conditions_category = [
    df['Summarize'].str.contains('oficina|local|oficinas', flags=re.IGNORECASE, na=False),
    df['Summarize'].str.contains('galpón', flags=re.IGNORECASE, na=False)
]
values = ['local', 'galpón']
default = 'vivienda'
df['category'] = np.select(conditions_category, values, default)
df

,Price,Location,Address,Has_photo,Features,Summarize,Description,Expensas,Link,currency,category
0,$ 1.500.000,"Monserrat, Capital Federal",Sarmiento al 1700,True,"[ 580 m² , 580 m² , 440 m² , 440 m² ]",Excelente Local en Venta - Alquiler,Excelente local comercial. Único en la zona. A...,None,https://www.zonaprop.com.ar/propiedades/clasif...,ARS,local
1,USD 12.913,"Centro / Microcentro, Capital Federal",Av. Corrientes al 800,True,"[ 586 m² , 586 m² , 586 m² , 586 m² , 3 ba...",Edificio Categoría a | Torre Odeon,"Edificio categoría A, desarrollado en 7 Subsue...",None,https://www.zonaprop.com.ar/propiedades/clasif...,USD,vivienda
2,$ 250.000,"Floresta, Capital Federal",Baradero 100. Entre Ramón lorenzo falcon y Raf...,True,"[ 40 m² , 40 m² , 38 m² , 38 m² , 1 amb. ,...",Departamento Monoambiente. | Baradero Al 100,Alquiler de Departamento monoambiente en Flore...,$ 33.000 Expensas,https://www.zonaprop.com.ar/propiedades/clasif...,ARS,vivienda
3,USD 35.533,"Catalinas, Capital Federal",Della Paolera al 200,True,"[ 1184 m² , 1184 m² , 1184 m² , 1184 m² , ...",Oficina Corporativa en Alquiler - Catalinas - ...,´200 Della Paolera´ es un edificio de oficinas...,$ 347.772 Expensas,https://www.zonaprop.com.ar/propiedades/clasif...,USD,local
4,USD 158.205,"Puerto Madero, Capital Federal","Cecilia Grierson 322, Puerto Madero",True,"[ 6022 m² , 6022 m² , 6022 m² , 6022 m² , ...","Oficinas en Alquiler | Dique Norte | Ed. Río, ...",Oportunidad única Dique Norte: Edificio Río - ...,None,https://www.zonaprop.com.ar/propiedades/clasif...,USD,local
...,...,...,...,...,...,...,...,...,...,...,...
18301,$ 250.000,"Caballito, Capital Federal",Avellaneda 0,True,"[ 33 m² , 33 m² , 30 m² , 30 m² , 1 amb. ,...",Avellaneda 0 Departamento 1 Ambiente Balcón 33...,Muy buen departamento de 1 ambiente al frente ...,$ 27.000 Expensas,https://www.zonaprop.com.ar/propiedades/clasif...,ARS,vivienda
18302,$ 250.000,"Colegiales, Capital Federal",Teodoro Garcia al 2800,True,"[ 30 m² , 30 m² , 2 amb. , 2 amb. , 1 dorm...",Departamento en Colegiales Muy Luminoso,Departamento muy bien ubicado en el barrio de ...,$ 20.000 Expensas,https://www.zonaprop.com.ar/propiedades/clasif...,ARS,vivienda
18303,USD 550,"Recoleta, Capital Federal",French 3000,True,"[ 40 m² , 40 m² , 40 m² , 40 m² , 2 amb. ,...","Departamento 2 Ambientes - Recoleta, Capital F...",Impecable 2 ambientes amoblado y con cochera f...,$ 109.432 Expensas,https://www.zonaprop.com.ar/propiedades/clasif...,USD,vivienda
18304,$ 265.000,"Parque Centenario, Capital Federal",DIAZ VELEZ 4200,True,"[ 35 m² , 35 m² , 28 m² , 28 m² , 1 amb. ,...",P/ Centenario Monoamb Excelente Buenas Condici...,Alquiler 24 meses parque centenario. buenas co...,$ 25.000 Expensas,https://www.zonaprop.com.ar/propiedades/clasif...,ARS,vivienda


In [30]:
def split_features(features):
    metros_cuadrados = np.nan
    ambientes = np.nan
    dormitorios = np.nan
    baños = np.nan
    cocheras = np.nan

    for feature in features:
        if 'm²' in feature:
            metros_cuadrados = int(feature.split()[0])
        elif 'amb.' in feature:
            ambientes = int(feature.split()[0])
        elif 'dorm.' in feature:
            dormitorios = int(feature.split()[0])
        elif 'baños' in feature:
            baños = int(feature.split()[0])
        elif 'coch.' in feature:
            cocheras = int(feature.split()[0])

    return pd.Series([metros_cuadrados, ambientes, dormitorios, baños, cocheras])

df[['metros_cuadrados', 'ambientes', 'dormitorios', 'baños', 'cocheras']] = df['Features'].apply(split_features)
df = df.drop("Features", axis=1)

In [31]:
df.value_counts("currency")

currency
USD          9743
ARS          8225
Consultar     338
Name: count, dtype: int64

In [32]:
df_ars = df.copy()
df_ars = df_ars[df_ars["currency"] == "ARS"].reset_index(drop = True)
df_ars['Price'] = df_ars['Price'].str.replace(r'[^\d\.]', '', regex=True)
df_ars['Price'] = df_ars['Price'].str.replace('.', '')
df_ars['Price'] = df_ars['Price'].astype(float)
df_ars['Expensas'] = df_ars['Expensas'].str.replace(r'[^\d\.]', '', regex=True)
df_ars['Expensas'] = df_ars['Expensas'].str.replace('.', '')
df_ars['Expensas'] = df_ars['Expensas'].astype(float)
df_ars["Expensas / alquiler"] = df_ars["Expensas"] / df_ars["Price"]
df_ars["Expensas / m2"] = df_ars["Expensas"] / df_ars["metros_cuadrados"]

In [33]:
df_usd = df.copy()
df_usd = df_usd[df_usd["currency"] == "USD"].reset_index(drop = True)
df_usd['Price'] = df_usd['Price'].str.replace(r'[^\d\.]', '', regex=True)
df_usd['Price'] = df_usd['Price'].str.replace('.', '')
df_usd['Price'] = df_usd['Price'].astype(float)
df_usd['Expensas'] = df_usd['Expensas'].str.replace(r'[^\d\.]', '', regex=True)
df_usd['Expensas'] = df_usd['Expensas'].str.replace('.', '')
df_usd['Expensas'] = df_usd['Expensas'].astype(float)

In [34]:
df_ars["Expensas"].isna().sum()

2436

In [35]:
df_ars

,Price,Location,Address,Has_photo,Summarize,Description,Expensas,Link,currency,category,metros_cuadrados,ambientes,dormitorios,baños,cocheras,Expensas / alquiler,Expensas / m2
0,1500000.0,"Monserrat, Capital Federal",Sarmiento al 1700,True,Excelente Local en Venta - Alquiler,Excelente local comercial. Único en la zona. A...,NaN,https://www.zonaprop.com.ar/propiedades/clasif...,ARS,local,440.0,NaN,NaN,NaN,NaN,NaN,NaN
1,250000.0,"Floresta, Capital Federal",Baradero 100. Entre Ramón lorenzo falcon y Raf...,True,Departamento Monoambiente. | Baradero Al 100,Alquiler de Departamento monoambiente en Flore...,33000.0,https://www.zonaprop.com.ar/propiedades/clasif...,ARS,vivienda,38.0,1.0,NaN,NaN,NaN,0.132000,868.421053
2,490000.0,"Belgrano, Capital Federal",11 de Septiembre 800,True,11 de Septiembre 800,Apto consultorio/oficinahermoso tres ambientes...,50000.0,https://www.zonaprop.com.ar/propiedades/clasif...,ARS,vivienda,80.0,3.0,1.0,NaN,NaN,0.102041,625.000000
3,610000.0,"Las Cañitas, Palermo",Migueletes al 900,True,Espectacular Monoambiente en La Mejor Zona de ...,Bellísimo monoambiente equipado full. Edificio...,NaN,https://www.zonaprop.com.ar/propiedades/clasif...,ARS,vivienda,41.0,1.0,NaN,NaN,NaN,NaN,NaN
4,700000.0,"Palermo Soho, Palermo",Paraguay 4445,True,Amoblado en Torre Quo Palermo - Seguridad 24 Has,amoblado y equipado ** alquiler temporario** t...,NaN,https://www.zonaprop.com.ar/propiedades/clasif...,ARS,vivienda,55.0,2.0,1.0,NaN,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8220,700000.0,"Recoleta, Capital Federal",Peña 2000 2°,True,Tres Ambientes en Recoleta Impecable,"Excelente tres ambientes Recoleta, un departam...",85000.0,https://www.zonaprop.com.ar/propiedades/clasif...,ARS,vivienda,70.0,3.0,2.0,NaN,NaN,0.121429,1214.285714
8221,250000.0,"Caballito, Capital Federal",Avellaneda 0,True,Avellaneda 0 Departamento 1 Ambiente Balcón 33...,Muy buen departamento de 1 ambiente al frente ...,27000.0,https://www.zonaprop.com.ar/propiedades/clasif...,ARS,vivienda,30.0,1.0,NaN,NaN,NaN,0.108000,900.000000
8222,250000.0,"Colegiales, Capital Federal",Teodoro Garcia al 2800,True,Departamento en Colegiales Muy Luminoso,Departamento muy bien ubicado en el barrio de ...,20000.0,https://www.zonaprop.com.ar/propiedades/clasif...,ARS,vivienda,30.0,2.0,1.0,NaN,NaN,0.080000,666.666667
8223,265000.0,"Parque Centenario, Capital Federal",DIAZ VELEZ 4200,True,P/ Centenario Monoamb Excelente Buenas Condici...,Alquiler 24 meses parque centenario. buenas co...,25000.0,https://www.zonaprop.com.ar/propiedades/clasif...,ARS,vivienda,28.0,1.0,1.0,NaN,NaN,0.094340,892.857143


In [40]:
df_ars = df_ars[
                (df_ars["category"] == "vivienda")
                & (df_ars["Expensas"]<=200_000)
                & (df_ars["Expensas"].notna())
                & (df_ars["Expensas"]>=10_000)
                & (df_ars["Price"]<=600_000)
                & (df_ars["Price"]>=10_000)
                & (df_ars["Expensas / alquiler"]<=1)
                & (df_ars["Expensas / m2"]<=10_000)
                & (df_ars["Expensas / m2"]>=10)
                ]
df_ars = df_ars.sort_values("Location")

In [48]:
df_ars.sort_values("Expensas / m2").iloc[1,7]

'https://www.zonaprop.com.ar/propiedades/clasificado/alclapin-hermoso-3-ambientes-en-alquiler-en-la-zona-de-53001048.html'

In [42]:
fig = go.Figure()
fig.add_trace(go.Box(x = df_ars["Location"],y = df_ars["Expensas / m2"]))
fig.update_layout(template = None, height = 600, margin = {'b':250})
fig.update_xaxes(type = "category")

In [14]:
fig = go.Figure()
fig.add_trace(go.Box(x = df_ars["Location"],y = df_ars["Price"]))
fig.update_layout(template = None, height = 600, margin = {'b':250})

In [15]:
tmp_df = df_ars.copy()
tmp_df = tmp_df.groupby("Location", as_index=False).median(numeric_only = True)
fig = go.Figure()
fig.add_trace(go.Scatter(x = tmp_df["Location"], y = tmp_df["Expensas / m2"], mode = "markers"))
fig.update_layout(template = None, margin = {"b":250})

In [16]:
df_usd = df_usd[
    (df_usd["metros_cuadrados"]<=60)
    & (df_usd["metros_cuadrados"].notna())
    & (df_usd["Price"] <= 2_000)
]

In [17]:
df_usd.sort_values("Price", ascending=True)[:20].iloc[18,5]

'Oficina en galeria de la flor - estado general buenosegundo piso al frenteen el corazon del microcentro. Muy cerca de todo. apto multiples destinos comerciales. lista para entrar. ascensor con cabina transparente. ambiente dividido en recepcion Y despacho Y/O lugar de trabajo. baño privado con ante-baño. pisos de parquet. Artefactos luminicos. un equipo individual de aire acondicionado tipo splitabl $ 800 - aysa $ 800. - matricula cucicba N° 2149carlos hugo rodriguez'

In [18]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = df_usd["Location"],y = df_usd["Price"], mode = "markers"))
fig.update_layout(template = None, height = 600, margin = {'b':250})

In [19]:
tmp_df = df_usd.copy()
tmp_df = tmp_df.groupby("Location", as_index=False).median(numeric_only = True)
fig = go.Figure()
fig.add_trace(go.Scatter(x = tmp_df["Location"], y = tmp_df["Price"], mode = "markers"))
fig.update_layout(template = None, margin = {"b":250}, title_text = "Costo de alquiler en USD. Fuente Zonaprop",
                  )

In [20]:
fig = go.Figure()
fig.add_trace(go.Box(y = df_usd["Price"], x = df_usd["Location"]))
fig.update_layout(template = None, margin = {"b":250}, title_text = "Costo de alquiler en USD. Fuente Zonaprop",
                  height = 1_000)
fig.show()